# Limpieza data raw "correcta"

- Se toma la data raw "correcta", la data preprocessed, que como dice el nombre es la data raw correguido valores nulos por fallas de conexión con PI, setear segundos a cero, etc.

- **Limpiezas hechas**:
    - Suavizar
    - Timegap
    - Pivot data
    - Eliminar PGP

    
- **Cambios con respecto a la limpieza ORIGINAL DEL RECOMENDADOR**:
    - Se suaviza solo con 2 periodos como ventanas en lugar de 5
    - Se utiliza el valor promedio del timegap para suavizar en lugar de un timegap dinámico (simplicar). Punto de observación al inicio del proceso
    - Se guarda en una columna el valor actual de tag y en otra columna el tag desplazado (CREACIÓN NUEVA NO EXISTENTE ANTES)

-------
**DATA**:
- INPUT: "data_raw_preprocessed.pkl"
- OUTPUT: "data_raw_processed.pkl"

## Root folder and read env variables

In [1]:
import os
# fix root path to save outputs
actual_path = os.path.abspath(os.getcwd())
list_root_path = actual_path.split('\\')[:-1]
root_path = '\\'.join(list_root_path)
os.chdir(root_path)
print('root path: ', root_path)

root path:  D:\github-mi-repo\Optimization-Industrial-Process


In [2]:
import os
from dotenv import load_dotenv, find_dotenv # package used in jupyter notebook to read the variables in file .env

""" get env variable from .env """
load_dotenv(find_dotenv())

""" Read env variables and save it as python variable """
PROJECT_GCP = os.environ.get("PROJECT_GCP", "")

## RUN

In [3]:
import pandas as pd
import numpy as np
import datetime as dt
import json
import pickle
from sklearn.pipeline import Pipeline
import sys
import os
import matplotlib.pyplot as plt
import gcsfs

import warnings
warnings.filterwarnings("ignore")

from sklearn.base import BaseEstimator, TransformerMixin

### 0. Funciones Auxiliares

In [4]:
# funciones auxiliares
def load_all_parameterstags_tagclassification(model_name):
    """
    Read a dictionary with all parameters filtered by model (d0eop, d1d2, d2, etc) located in TagClassification 
    """
    path_json = 'config/params.json'
    with open("{path}".format(path=path_json)) as json_file:
        tag_classification_pars = json.load(json_file)

    return tag_classification_pars[model_name]

In [5]:
# define name of model - transversal model for this example
general_params_models = 'blanqueo_santafe_all'

### 1. Leer data raw - datalake
### 1. Read data raw datalake - preprocessed
- Data get in the previous notebook
- Data without nulls - filled in previous step for problems in upload data, no conextion PI-datalake, etc

In [6]:
path_raw_data = f'artifacts/data/data_raw_preprocessed.pkl'
basic_preprocessed_data = pd.read_pickle(path_raw_data)
basic_preprocessed_data.head(3)

,datetime,Tag,PV
0,2021-01-01 00:00:00,230AIT446.PNT,11.55654
1,2021-01-01 00:05:00,230AIT446.PNT,11.55354
2,2021-01-01 00:10:00,230AIT446.PNT,11.55110


In [7]:
print('primera fecha: ', basic_preprocessed_data['datetime'].min())
print('ultima fecha: ', basic_preprocessed_data['datetime'].max())

primera fecha:  2021-01-01 00:00:00
ultima fecha:  2023-01-02 00:00:00


In [8]:
# validate number of nulls
df_aux = basic_preprocessed_data.pivot(index='datetime', columns='Tag', values='PV')
df_aux.isnull().sum().sum()

0

## Continuar los pasos de limpieza después de obtener data raw correcta sin problemas de valores nulos

### 5. Suavizar Data
En el json está el parámetro de periodos que se utilizan para suavizar

In [9]:
class smooth_data(BaseEstimator,TransformerMixin):
    '''
    Suavizar la data. 
    - Actualmente suavizar toda la data (excepto los tags de tiempo de retención) de acuerdo a un parámetro fijo
    - Se suaviza cuando está toda la data con valores y NO HAY NULOS, así evito que se expandan los nulos al aplicar el rolling
    
    - Obs: recibe la data raw, pivotear, suaviza, vuelve
    '''
    
    def __init__(self, freq_rolling_smooth_data, timegap_tag_list):
        super(smooth_data,self).__init__()
        self.freq_rolling_smooth_data = freq_rolling_smooth_data # freq con la que se va suavizar
        self.timegap_tag_list = timegap_tag_list # listado de tags de tiempos de retención

    def fit(self,DataFrame):
        return self

    def transform(self,DataFrame):
        print('\napplying smooth_data...')
        
        # pivotear data
        print('Tamaño data raw: ', DataFrame.shape)
        DataFrame = DataFrame.pivot(index='datetime', columns='Tag', values='PV')

        # definir tags que se van a suavizar -> todos excepto los tag de timegap (esos datos tienen su propio tratamiento en otra función)
        list_tags_to_smooth = list(set(DataFrame.columns.tolist()) - set(self.timegap_tag_list ))
        
        # suavizar la data y borrar nulos generados
        DataFrame[list_tags_to_smooth] = DataFrame[list_tags_to_smooth].rolling(self.freq_rolling_smooth_data).mean()
        DataFrame = DataFrame.dropna()
        print('Cantidad de datos a borrarse por suavizar la data(primeros valores): ', (self.freq_rolling_smooth_data - 1) * DataFrame.shape[1])
        
        
        # unpivot de la data porque el timegap dinámico está armado en base a la estructura de la data raw
        DataFrame = DataFrame.reset_index()
        DataFrame = pd.melt(DataFrame, id_vars = ['datetime'])
        DataFrame.rename(columns = {'value': 'PV'}, inplace = True)
        print('Tamaño data luego de suavizar: ', DataFrame.shape)
        
        return DataFrame

In [10]:
# IMPORTANTE
# ventana filtro ma - AHORA ES DE 2 PERIODOS COMO VENTANA (en lugar de 5 periodos como tiene la limpiezaz original)
selected_model_name = 'blanqueo_santafe_all'
tag_classification_pars = load_all_parameterstags_tagclassification(model_name = general_params_models)
moving_average_all_tags = tag_classification_pars['cleaning_pars']['moving_average_all_data']
moving_average_all_tags

2

In [11]:
# read master tag table
path_MaestroEtapas = 'config/MaestroEtapas.xlsx'
MaestroEtapas = pd.read_excel(path_MaestroEtapas)

# list of tags wuith values of timegap - this tags has its own process of cleaning
timegaptag_list = list(MaestroEtapas.dropna()['TAG_TIEMPO_RESIDENCIA'])

In [12]:
# instancia de la clase
smooter = smooth_data(freq_rolling_smooth_data = moving_average_all_tags, 
                      timegap_tag_list = timegaptag_list
                     )

# transform
preprocessed_data = smooter.transform(basic_preprocessed_data)


applying smooth_data...
Tamaño data raw:  (12842269, 3)
Cantidad de datos a borrarse por suavizar la data(primeros valores):  61
Tamaño data luego de suavizar:  (12842208, 3)


### 6. TimeGap PROMEDIO
- En la limpieza original se utiliza los timegap dinámicos (se utilizan tiempos de residencia como función de la producción y el tamaño del tanque). Esto tiene diferentes problemas al generar muchos nulos

- En este ejemplo sencillo, se obviará los timegap dinámicos y se utilizará un TIMEGAP PROMEDIO (donde este valor promedio se obtuvo en combinación con los tiempos de retención promedio que manejan los ingenieros de procesos y validado a través de los tags de tiempo de renteción obteniendo el promedio de cada uno de lo valores de tiempo de residencia)

- Al utilizar simplemente un desplazamiento promedio no se generan nulos y es relativamente cercano a la realidad (por lo menos cuando el proceso es estable el cual cual debería ser la mayor parte del tiempo)

- Para hacer los desplazamiento de la data me voy a parar AL INICIO DEL PROCESO en la etapa D0 (en una primera instancia no se va a considerar la etapa ácida)

#### 6.1 Auxiliar. Crear diferentes tags de producción para ser desplazados
- Originalmente existen los tags de producción a la entrada de blanqueo (etapa A), entrada de eop y bypass.
- Se van a crear tags de producción para la etapa D0, D1 y P (simplemente tomar la producción del blanqueo a la entrada y crear las variables de producción a la entrada de las etapas D0, D1 y P, las que posteriormente se desplazaran de acuerdo al tiempo de residencia 

In [13]:
def crear_tags_produccion_cada_etapa(df):
    '''
    Crear tag de producción en las etapas que no existe directamente el tag de producción: D0, D1, P
    '''
    # pivotear data
    df_pivot = df.pivot(index='datetime', columns='Tag', values='PV')

    """
    ######### DISCOVERY #########
    ### DISCOVERY - DIFERENCIA ENTRE LOS VALORES DE (PRODUCCIÓN A LA ENTRADA DE BLANQUEO) VS (PRODUCCIÓN ENTRADA E0P + BYPASS)
    ### DISCOVERY - SE OBSERVA QUE LA DIFERENCIA ENTRE AMBAS VOLARES RONDA ENTRE 20 A 70 ADT

    list_tags_produccion = ['240FI020A.PNT', '240FI020B.PNT', '240FI108A.PNT']
    df_pivot['prod_eop_bypass'] = df_pivot['240FI020B.PNT'] + df_pivot['240FI108A.PNT']
    df_pivot['diff_prod_d0_eopbypass'] = df_pivot['240FI020A.PNT'] - df_pivot['prod_eop_bypass']
    print('\nDescribe diferencia')
    print(df_pivot['diff_prod_d0_eopbypass'].describe(percentiles = [0.05, 0.1, 0.25, 0.5, 0.75, 0.90, 0.95]))
    print('\nHistograma diferencia')
    df_pivot['diff_prod_d0_eopbypass'].hist()
    """

    # crear tags de producción - aún no se desplaza la data - en el maestro tag está definido el tag con el nombre que se va a
    # crear para desplazar los valores de producción a su torre correspondiente
    df_pivot['calc_prod_d0'] = df_pivot['240FI020A.PNT']
    df_pivot['calc_prod_d1'] = df_pivot['240FI020A.PNT']
    df_pivot['calc_prod_p'] = df_pivot['240FI020A.PNT']

    # unpivot
    df_unpivot = pd.melt(df_pivot.reset_index(), id_vars = ['datetime'])
    df_unpivot.columns = ['datetime', 'Tag', 'PV']
    
    return df_unpivot

In [14]:
preprocessed_data = crear_tags_produccion_cada_etapa(preprocessed_data)

#### 6.2 Aplicar timegap promedio
- Se necesita archivo: Maestro Etapas que tiene las etapas y sus tiempos de retención promedio
- Se necesita archivo: Maestro Tags que tiene la clasificación de los tags en cada una de las etapas

In [15]:
# auxiliar
def get_list_stages_to_calculate_timegap(stage_start, MaestroEtapas):
    '''
    - Definir una lista de los stages que se van a considerar para aplicar timegap.
    - El input contiene "stage_start" que indica desde qué stage se quiere considerar. Por ejemplo, si fuera D1, quiero solo
    considerar los stage desde el D1 en adelante (incluyendolo).
    '''
    
    # listado completo de stages desde el maestro etapas
    list_all_stages = MaestroEtapas['ETAPA'].tolist()
    
    # obtener solo el listado de stages desde la etapa que me interesa
    list_stages = []
    mark = False # marca para indicar que se encontró la etapa desde la que se va a considerar timegap
    for stage in list_all_stages:

        # si se llega al stage desde que se quiere aplicar el timegap hacia adelante, la marca cambia a True
        if stage == stage_start:
            mark = True

        # siolo si la marca es True, se considera guardar el stage en el listado de stages que se van a considerar
        if mark == True:
            list_stages.append(stage)
            
    return list_stages

In [16]:
def apply_timegap_mean(df):
    '''
    Aplicar el timegap Promedio a toda la data DESDE LA ETAPA ÁCIDA HACIA ADELANTE
    '''
    
    """ pivotear data """
    df_pivot = df.pivot(index='datetime', columns='Tag', values='PV')

    """ read MaestroTags """
    path_MaestroTags = f'config/MaestroTags.xlsx'
    MaestroTags = pd.read_excel(path_MaestroTags)

    """ read MaestroEtapas """
    path_MaestroEtapas = f'config/MaestroEtapas.xlsx'
    MaestroEtapas = pd.read_excel(path_MaestroEtapas)

    # obtener listado de stages para timegap
    #list_stages_to_timegap = get_list_stages_to_calculate_timegap(stage_start = 'A', MaestroEtapas = MaestroEtapas)
    list_stages_to_timegap = MaestroEtapas['ETAPA'].tolist()

    """
    validar que todos los datos del dataframe estén en el maestro tags (asegurarse que todos los tags tiene asignado una etapa)
    y filtrar maestro tags tenga solo los tags presentes en la data
    """
    list_tags_data = df_pivot.columns.tolist()
    list_tags_maestro_tags = MaestroTags['TAG'].tolist()
    tags_not_in_maestro_tags = [tag for tag in list_tags_data if tag not in list_tags_maestro_tags]
    if tags_not_in_maestro_tags == []:
        print('Todos los datos del dataframe están en el maestro tags. Por lo tanto todos los tags tienen asignada una etapa')
        MaestroTags = MaestroTags.query(f'TAG == {list_tags_data}')

    """
    mover la data de acuerdo a los tiempos de residencia promedio
    """

    # inicializar
    timegap_acc = 0
    df_final = pd.DataFrame(index = df_pivot.index)

    # para cada uno de los stages...
    for stage in list_stages_to_timegap:
        print('\n\n')
        print('CALCULANDO TIEMPOS DE RESIDENCIA ETAPA:', stage)
        print('Timegap acumulado (observaciones): ', timegap_acc)


        ############## DEFINIR LISTADO DE TAGS ##############
        # filtrar el maestro tags solo por los tags que están en la etapa que se está calculando
        list_tags_stage = MaestroTags[MaestroTags['ETAPA'] == stage]['TAG'].tolist()
        print('Listado de tags del stage: ', list_tags_stage)

        # listado con el sufijo "_visto" corresponden al valor del tag visto en la realidad
        names_tag_visto = [tag + '_visto' for tag in df_pivot[list_tags_stage].columns]

        # listado con el sufijo "_real" corresponden al valor del tag desplazado de acuerdo al tiempo de residencia
        names_tag_real = [tag + '_real' for tag in df_pivot[list_tags_stage].columns]


        ############## CREAR DATASET VISTO(DATA EN EL MOMENTO ACTUAL) Y DATASET REAL (DATA DESPLAZADA DE ACUERDO AL TIMEGAP) ##############
        # data vista agregarle prefijo "visto"
        df_pivot_visto = df_pivot[list_tags_stage].copy()
        df_pivot_visto.columns = names_tag_visto

        # data real agregarle prefijo "real"
        df_pivot_real = df_pivot[list_tags_stage].copy()
        df_pivot_real.columns = names_tag_real

        # data real desplazarla de acuerdo al tiempo de residencia
        df_pivot_real = df_pivot_real.shift(-timegap_acc)


        ############## DATASET VISTO Y REAL GUARDARLOS Y TERMINAR ITERACIÓN DEL STAGE ACTUAL ##############
        # agregar data real y data vista al dataframe final
        df_final = df_final.merge(df_pivot_visto, left_index = True, right_index = True)
        df_final = df_final.merge(df_pivot_real, left_index = True, right_index = True)

        # calcular el valor de tiempo de residencia acumulado al terminar esta stage - en base a observaciones de 5 minutos
        timegap_acc += int(MaestroEtapas[MaestroEtapas['ETAPA'] == stage]['TIEMPO_RESIDENCIA_PROMEDIO'].values[0] / 5)


    df_final_unpivot = pd.melt(df_final.reset_index(), id_vars = ['datetime'])
    df_final_unpivot.columns = ['datetime', 'Tag', 'PV']
    return df_final_unpivot

In [17]:
preprocessed_data = apply_timegap_mean(preprocessed_data)

Todos los datos del dataframe están en el maestro tags. Por lo tanto todos los tags tienen asignada una etapa



CALCULANDO TIEMPOS DE RESIDENCIA ETAPA: A
Timegap acumulado (observaciones):  0
Listado de tags del stage:  ['240FI020A.PNT', 'S220ALDP010', '230AIT446.PNT', '240LIT010.PNT', '240AIC022.MEAS', '240TIC023.MEAS', '240FY024A.RO01', '240FIC024.MEAS', '276CLO2_LGA.RO02', 'S76ALE017', '240FI020A_HRS_TORRE.C']



CALCULANDO TIEMPOS DE RESIDENCIA ETAPA: D0
Timegap acumulado (observaciones):  18
Listado de tags del stage:  ['calc_prod_d0', '240AIT063B.PNT', '240AIT063A.PNT', 'S276PER002', 'SSTRIPPING015', '240FY050.RO02', '240FIC110.MEAS', '240FY039.RO01', '240FIC440.MEAS', '240FI020A_HRS_DO.C']



CALCULANDO TIEMPOS DE RESIDENCIA ETAPA: EOP
Timegap acumulado (observaciones):  20
Listado de tags del stage:  ['240FI020B.PNT', '240TI139.PNT', '240AIC126.MEAS', '240FY11PB.RO01', '240FY118B.RO01', '240FY107A.RO01', '240FIC116.MEAS', '240FIC107.MEAS', '240FIC118.MEAS', 'S240ALDP031', 'S24

### 7. Pivotear data

In [18]:
class Pivot(BaseEstimator,TransformerMixin):
    '''
    A class used to transform the data with tags in vertical to data with tags horizontal(in columns)
    
    Input:
        original_datetime	ETAPA	Tag	            PV	        datetime
    0	2021-10-01 00:20:00	EOP	    240AIC126.MEAS	11.199728	2021-10-01 13:55:00
    1	2021-10-01 00:25:00	EOP	    240AIC126.MEAS	11.210860	2021-10-01 14:00:00
    2	2021-10-01 00:30:00	EOP	    240AIC126.MEAS	11.217164	2021-10-01 14:05:00
    
    Si se pivotea la data original (original_datetime), no existen valores nulos
    Si se pivotea la data aplicado el timegap, si existen valores nulos
    '''
    def __init__(self):
        super(Pivot,self).__init__()

    def fit(self,DataFrame):
        return self

    def transform(self,DataFrame):
        print('\naplicando pivot...')
        
        # hacer trnasformacion
        DataFrame = DataFrame.pivot_table(index = "datetime", columns = "Tag", values = "PV")
        
        #info
        print('\n Tamaño dataframe: ', DataFrame.shape)
        print('\nCantidad de nulos (estos nulos se generan a aplicar el timegap): ', DataFrame.isnull().sum())
        print('\n% de nulos (estos nulos se generan a aplicar el timegap): ', 100 * (DataFrame.isnull().sum() / DataFrame.shape[0]))
        return DataFrame

In [19]:
# instancia de la clase
pivoter = Pivot()

# transformar data
preprocessed_data = pivoter.transform(preprocessed_data)


aplicando pivot...

 Tamaño dataframe:  (210528, 128)

Cantidad de nulos (estos nulos se generan a aplicar el timegap):  Tag
230AIT446.PNT_real       0
230AIT446.PNT_visto      0
240AIC022.MEAS_real      0
240AIC022.MEAS_visto     0
240AIC126.MEAS_real     20
                        ..
calc_prod_d0_visto       0
calc_prod_d1_real       30
calc_prod_d1_visto       0
calc_prod_p_real        48
calc_prod_p_visto        0
Length: 128, dtype: int64

% de nulos (estos nulos se generan a aplicar el timegap):  Tag
230AIT446.PNT_real      0.00000
230AIT446.PNT_visto     0.00000
240AIC022.MEAS_real     0.00000
240AIC022.MEAS_visto    0.00000
240AIC126.MEAS_real     0.00950
                         ...   
calc_prod_d0_visto      0.00000
calc_prod_d1_real       0.01425
calc_prod_d1_visto      0.00000
calc_prod_p_real        0.02280
calc_prod_p_visto       0.00000
Length: 128, dtype: float64


### 8. Eliminar datos de PGP

In [20]:
class DropMaintenanceDates(BaseEstimator,TransformerMixin):
    '''
    A class to delete the dates when the plant was not operative
    '''
    
    def __init__(self,MaintenanceDataFrame,beginDateCol,endDateCol,**kwargs):
        super(DropMaintenanceDates,self).__init__()
        self.MaintenanceDataFrame = MaintenanceDataFrame 
        self.beginDateCol = beginDateCol
        self.endDateCol = endDateCol
        self.kwargs = kwargs

    def fit(self,DataFrame):
        return self

    def transform(self,DataFrame):
        print('\naplicando limpiezas fechas de mantención - PGP ...')
        print('Tamaño data: ', DataFrame.shape)
        
        for i, row in self.MaintenanceDataFrame.iterrows():
            DropIndex=DataFrame.loc[row[self.beginDateCol]:row[self.endDateCol]].index
            DataFrame.drop(DropIndex,inplace=True)
            
        #info
        print('Tamaño data luego de borrar fechas de matención: ', DataFrame.shape)
        print('\n% de nulos hasta el momento: ', 100 * (DataFrame.isnull().sum() / DataFrame.shape[0]))
        return DataFrame

In [21]:
def create_maintenance_df(beginning_date, ending_date):
    maintenance_df = pd.DataFrame()
    maintenance_df["beginDate"] = beginning_date
    maintenance_df["endDate"] = ending_date
    return maintenance_df

In [22]:
# parámetros
tag_classification_pars = load_all_parameterstags_tagclassification(model_name = general_params_models)
beginning_date = tag_classification_pars['maintenance_dates']['beginDate']
ending_date =  tag_classification_pars['maintenance_dates']['endDate']
maintenance_df = create_maintenance_df(beginning_date, ending_date)

pars_drop_maintenance_dates = {
        "MaintenanceDataFrame": maintenance_df,
        "beginDateCol": "beginDate",
        "endDateCol": "endDate",
    }

In [23]:
# instancia de la clase
droper_maintenance_dates = DropMaintenanceDates(**pars_drop_maintenance_dates)

# transformar dataframe
processed_data = droper_maintenance_dates.transform(preprocessed_data)


aplicando limpiezas fechas de mantención - PGP ...
Tamaño data:  (210528, 128)
Tamaño data luego de borrar fechas de matención:  (197568, 128)

% de nulos hasta el momento:  Tag
230AIT446.PNT_real      0.000000
230AIT446.PNT_visto     0.000000
240AIC022.MEAS_real     0.000000
240AIC022.MEAS_visto    0.000000
240AIC126.MEAS_real     0.010123
                          ...   
calc_prod_d0_visto      0.000000
calc_prod_d1_real       0.015185
calc_prod_d1_visto      0.000000
calc_prod_p_real        0.024295
calc_prod_p_visto       0.000000
Length: 128, dtype: float64


### 9. GUARDAR PKL PROCESSED

In [24]:
# save data pkl cloud
path_raw_data_processed = 'artifacts/data/data_raw_processed.pkl'
with open(path_raw_data_processed, "wb") as output:
    pickle.dump(processed_data, output)
    output.close()